In [3]:
import pytraj as pt
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import numpy as np
from tqdm.notebook import trange, tqdm
import pickle

In [4]:
with open(f"RUNS.pkl", "rb") as f:
            RUNS = pickle.load(f)

## read in domains 
with open ('5gae_domains.pkl', 'rb') as fp:
    domains = pickle.load(fp)

In [11]:
with open("RUNS.pkl", "rb") as f:
    RUNS_globalfield = pickle.load(f)

In [ ]:
for key in tqdm(["posf", "negf"]):
    RUNS_globalfield[key]["pytrj"] = pt.iterload(
                    ["../../" + RUNS_globalfield[key]["folder"] + f"/{i}/prot_center_mol_fit_{i}.xtc" for i in [1,2,3]], 
                     "/itpstore/DK_Nanocell/Horvath/I__SecYEG/J__5GAE/16__Efield_protein_only/step1_pdbreader.pdb", 
                     stride = 1)
    RUNS_globalfield[key]["pytrj"] = RUNS_globalfield[key]["pytrj"].superpose("@CA")

In [5]:
for key in [key for key in RUNS if key != "nof"]:
    #if key != "nof":
        del RUNS[key]

In [6]:
RUNS["posf_inv"] = {}
RUNS["posf_inv"]["folder"] = "/itpstore/DK_Nanocell/Horvath/I__SecYEG/J__5GAE/16__Efield_protein_only/A__pept_bond_only/A__200mV/1__posf/"

RUNS["negf_inv"] = {}
RUNS["negf_inv"]["folder"] = "/itpstore/DK_Nanocell/Horvath/I__SecYEG/J__5GAE/16__Efield_protein_only/A__pept_bond_only/A__200mV/2__negf/"

In [8]:
for key in ["posf_inv", "negf_inv"]:
    RUNS[key]["pytrj"] = pt.iterload(RUNS[key]["folder"] + "cat_prot_center_mol.xtc",
                                    "/itpstore/DK_Nanocell/Horvath/I__SecYEG/J__5GAE/16__Efield_protein_only/step1_pdbreader.pdb")
    RUNS[key]["pytrj"] = RUNS[key]["pytrj"].superpose("@CA")
    print(RUNS[key]["pytrj"])

pytraj.TrajectoryIterator, 100004 frames: 
Size: 20.197842 (GB)
<Topology: 9036 atoms, 573 residues, 0 mols, non-PBC>
           
pytraj.TrajectoryIterator, 100004 frames: 
Size: 20.197842 (GB)
<Topology: 9036 atoms, 573 residues, 0 mols, non-PBC>
           


In [10]:
key="nof"
xtc = "prot_center_mol_fit_"
RUNS[key]["folder"] = "/itpstore/DK_Nanocell/Horvath/I__SecYEG/J__5GAE/5__cMD/"

RUNS[key]["pytrj"] = pt.iterload([RUNS[key]["folder"] + f"/{i}/{xtc}{i}.xtc"
                                      for i in [1,2,3,4,5,6]
                                 ], 
                                 #RUNS[key]["folder"] + "step1_pdbreader.pdb", 
                                 "/itpstore/DK_Nanocell/Horvath/I__SecYEG/J__5GAE/5__cMD/protein.pdb",
                                 stride = 1)
RUNS[key]["pytrj"] = RUNS[key]["pytrj"].superpose("@CA")
RUNS[key]["pytrj"]                   

pytraj.TrajectoryIterator, 82506 frames: 
Size: 16.663765 (GB)
<Topology: 9036 atoms, 573 residues, 0 mols, non-PBC>
           

In [29]:
RUNS["posf"] = RUNS_globalfield["posf"]
RUNS["negf"] = RUNS_globalfield["negf"]

In [14]:
list(RUNS)

['nof', 'posf_inv', 'negf_inv', 'posf', 'negf']

# Distance from pore center

In [15]:
def dist_from_pore_center_of_geom(key, skip=10):
    CA_xyz = RUNS[key]["pytrj"][::skip]["@CA"].xyz ## ca positions for entire protein
    print("CA loaded")
    pore_cog = pt.center_of_geometry(RUNS[key]["pytrj"][::skip], mask = ":86,405,278,191") ## pore center
    print("pore loaded")
    xy_dist = CA_xyz[:,:,:2] - pore_cog[:,:2].reshape(len(pore_cog),1, 2) # distance from center as vector in xy 
    #print(key, " done")
    return np.linalg.norm(xy_dist,axis=2) # take norm over the xy axis


In [ ]:
for key in tqdm(RUNS):
    RUNS[key]["pore_dist_xy"] = dist_from_pore_center_of_geom(key, skip=1)

In [31]:
def delta_dist(key, refkey="nof", ddict=RUNS):
    return ddict[key]["pore_dist_xy"].mean(axis=0) - ddict[refkey]["pore_dist_xy"].mean(axis=0)

In [ ]:
from scipy.stats import bootstrap
for key in tqdm(list(RUNS)):
    RUNS[key]["pore_dist_xy_boots_std"] = bootstrap(
              (RUNS[key]["pore_dist_xy"], ),
              statistic=np.std,
              axis=0,
              n_resamples=100,
              confidence_level=0.95,
              random_state=1, 
              method='percentile',
         ).confidence_interval

In [34]:
with open(f'backbone_only_data.pkl', 'wb') as f:
    pickle.dump({key: 
                    {
                        subk: RUNS[key][subk] for subk in RUNS[key] if subk!="pytrj"
                    } for key in RUNS},
                f)